In [14]:
import pandas as pd
data1 = pd.read_csv('output/코사인용2.csv')
data2 = pd.read_csv('output/냉장고재료test용.csv')
s = ' '.join(data2['ingredient'])
print(s)
data1.loc[0] = ['냉장고재료', '',s , '', '', '', '', '', '']
print(data1['name'])
data1.to_csv('output/코사인용2.csv', index = False)

양파 당근 계란 마늘 무 양배추 깻잎 파프리카
0                          냉장고재료
1            새콤달콤 파래무침 만들기 황금레시피
2       오늘뭐먹지? 무수분된장찜 - 초간단 메인반찬
3                       백종원 두부조림
4                   국물이 있는 두부조림 
                  ...           
7863                이쁘고 맛난 양배추쌈 
7864                  콩나물불고기 만들기
7865                  오이 피클 샌드위치
7866            아기를 위한 대구크림소스그라탕
7867                     미역 오이냉국
Name: name, Length: 7868, dtype: object


In [27]:
print(type(data1['ingredient'][4]))
print(type(data1['seasoning']))
data1 = data1.astype({'seasoning': 'str'})
print(type(data1['seasoning'][4]))
text = data1['ingredient'][4] + data1['seasoning'][4]
data1['ingredient'][4]
# print(data1['seasoning'][4])
# print(text)

<class 'str'>
<class 'pandas.core.series.Series'>
<class 'str'>


'\ufeff부침용 두부 1모, 당근, 파, 고추, 양파, 다진마늘, 고춧가루, 들기름, 멸치육수 (or 물), 집간장, 액젓, 설탕'

In [37]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# data = pd.read_csv('output/코사인용.csv')
data = pd.read_csv('output/코사인용2.csv')
# data = pd.read_csv('output/코사인용2.csv', low_memory=False)

# data['ingredient'] = data['ingredint_name'].fillna('')
# print(type(data['ingredient']))
tfidf = TfidfVectorizer()
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['ingredient_name'])

print(tfidf_matrix.shape)
# overview 열에 대해서 tf-idf를 수행했습니다. 20,000개의 영화를 표현하기위해 총 47,487개의 단어가 사용되었음을 보여주고 있습니다.
# 이제 코사인 유사도를 사용하면 바로 문서의 유사도를 구할 수 있습니다.

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# 코사인 유사도를 구합니다.

indices = pd.Series(data.index, index=data['name'])
print(indices.head())
# 영화의 타이틀과 인덱스를 가진 테이블을 만듭니다. 이 중 5개만 출력해보도록 하겠습니다.
# 이 테이블의 용도는 영화의 타이틀을 입력하면 인덱스를 리턴하기 위함입니다.


# 이제 선택한 영화에 대해서 코사인 유사도를 이용하여, 가장 overview가 유사한 10개의 영화를 찾아내는 함수를 만듭니다.
def get_recommendations(food, cosine_sim=cosine_sim):
    # 선택한 영화의 타이틀로부터 해당되는 인덱스를 받아옵니다. 이제 선택한 영화를 가지고 연산할 수 있습니다.
    idx = indices[food]

    # 모든 영화에 대해서 해당 영화와의 유사도를 구합니다.
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 영화들을 정렬합니다.
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    print(sim_scores[:9])

    # 가장 유사한 10개의 영화를 받아옵니다.
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개의 영화의 인덱스를 받아옵니다.
    food_indices = [i[0] for i in sim_scores]
    print(type(food_indices))
    print(food_indices[0])
    print(type(food_indices[0]))
    
    # 가장 유사한 10개의 영화의 제목을 리턴합니다.
    return data['name'].iloc[food_indices]

get_recommendations('냉장고재료')

(7868, 2295)
name
냉장고재료                       0
새콤달콤 파래무침 만들기 황금레시피         1
오늘뭐먹지? 무수분된장찜 - 초간단 메인반찬    2
백종원 두부조림                    3
국물이 있는 두부조림                 4
dtype: int64
[(0, 0.9999999999999998), (6433, 0.6404098912811858), (7326, 0.6404098912811858), (3208, 0.6370544133157889), (1875, 0.6357949789882995), (6134, 0.6024825738009586), (7194, 0.5824991839576408), (893, 0.581399188832226), (1141, 0.581399188832226)]
<class 'list'>
6433
<class 'int'>


6433                             아삭한 양배추무침
7326    [양배추깻잎피클과 양배추김치] 양배추 한 통 야무지게 활용하기
3208                              매콤한 순대볶음
1875                          백종원 제육볶음 레시피
6134                          알록달록 예쁜 햄볶음밥
7194                  호텔식 부럽지 않은 오므라이스 만들기
893                      초스피드 반찬, 전자렌지 깻잎찜
1141                              밥도둑 깻잎김치
2380                              깻잎지와 깻잎찜
1123                            순대볶음 황금레시피
Name: name, dtype: object

In [51]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

# data = pd.read_csv('output/코사인용.csv')
data = pd.read_csv('output/코사인용2.csv')
# data = pd.read_csv('output/코사인용2.csv', low_memory=False)

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(data['ingredient_name'])

print(tfidf_matrix.shape)

# 코사인유사도
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

# 인덱스대신 요리명을 활용하기 위한 테이블
indices = pd.Series(data.index, index=data['name'])

def get_recommendations(food, cosine_sim=cosine_sim):
    # 인덱스 불러오기 냉장고재료의 경우 값은 0으로 고정
    # 다른 레시피와 유사한 레시피의 경우 인덱스 받아옴
    idx = indices[food]

    # 모든레시피와 유사도 검사
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 레시피 정렬
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
#     print(sim_scores[:9])

    # 유사도가 높은 30개 레시피 (0번은 자기자신)
    sim_scores = sim_scores[1:31]

    # 유사도가 높은 30개의 레시피 인덱스 추출
    food_indices = [i[0] for i in sim_scores]
#     print(food_indices)
    
    # 인덱스를 활용하여 30개의 레시피를 조회수 순으로 재정렬
    data2 = data.iloc[food_indices]
    data2.sort_values(by='views', ascending=False, inplace=True)
    
    # 30개의 유사한 레시피중 조회수가 높은 10개의 레시피 추출
    new_indices = data2['Unnamed: 0'].values.tolist()[:10]
    return data['name'].iloc[new_indices]

get_recommendations('냉장고재료')


(7868, 2295)


C:\Users\wima9\anaconda3\envs\pydatavenv\lib\site-packages\ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


893     초스피드 반찬, 전자렌지 깻잎찜
1123           순대볶음 황금레시피
1875         백종원 제육볶음 레시피
7789         깻잎김치, 황금 레시피
3414         "굴소스야채볶음라면"♪
3474       촉촉한 중국풍 양배추햄덮밥
1229           김치제육볶음 만들기
1066       깻잎김치 담는 간단한 방법
1141             밥도둑 깻잎김치
169                 깻잎절임 
Name: name, dtype: object

In [50]:
df = pd.read_csv('output/냉장고재료test용.csv')
data = ' '.join(df['ingredient'].values)
print(type(data))
print(data)

<class 'str'>
양파 당근 계란 마늘 무 양배추 깻잎 파프리카


In [42]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.metrics.pairwise import linear_kernel

data = pd.read_csv('output/코사인용.csv')
# data = pd.read_csv('output/코사인용2.csv', low_memory=False)

# data['ingredient'] = data['ingredint_name'].fillna('')
# print(type(data['ingredient']))
hv = HashingVectorizer(n_features=300000)
tfidf = TfidfVectorizer(max_features=300000)
print(tfidf)
# overview에 대해서 tf-idf 수행
tfidf_matrix = tfidf.fit_transform(data['ingredient_name'])
hv_matrix = hv.fit_transform(data['ingredient_name'])

print(tfidf_matrix.shape)
print(hv_matrix.shape)

TfidfVectorizer(max_features=300000)
(96064, 7737)
(96064, 300000)


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
TfidfVectorizer?

Init signature:
TfidfVectorizer(
    *,
    input='content',
    encoding='utf-8',
    decode_error='strict',
    strip_accents=None,
    lowercase=True,
    preprocessor=None,
    tokenizer=None,
    analyzer='word',
    stop_words=None,
    token_pattern='(?u)\\b\\w\\w+\\b',
    ngram_range=(1, 1),
    max_df=1.0,
    min_df=1,
    max_features=None,
    vocabulary=None,
    binary=False,
    dtype=<class 'numpy.float64'>,
    norm='l2',
    use_idf=True,
    smooth_idf=True,
    sublinear_tf=False,
)
Docstring:     
Convert a collection of raw documents to a matrix of TF-IDF features.

Equivalent to :class:`CountVectorizer` followed by
:class:`TfidfTransformer`.

Read more in the :ref:`User Guide <text_feature_extraction>`.

Parameters
----------
input : {'filename', 'file', 'content'}, default='content'
    If 'filename', the sequence passed as an argument to fit is
    expected to be a list of filenames that need reading to fetch
    the raw content to analyze.

    If 'file', the

In [16]:
cosine_sim = linear_kernel(hv_matrix, hv_matrix)

RuntimeError: nnz of the result is too large